In [1]:
# imports
import time
import sklearn
import pandas as pd
import numpy
import multiprocess as mp

from PIL import Image, ImageOps, ImageEnhance
from sklearn.metrics import classification_report, accuracy_score
from skimage.feature import hog
from sklearn import svm
from matplotlib import cm


# directorios
train_dir = '../data/processed/train'
test_dir = '../data/processed/test'

train_csv_path = '../data/processed/train_split.txt'
test_csv_path = '../data/processed/test_split.txt'

train_fd_csv_path = '../data/processed/train_fd.txt'
test_fd_csv_path = '../data/processed/test_fd.txt'

train_hoged_imgs_csv_path = '../data/processed/train_hogged.txt'
test_hoged_imgs_csv_path = '../data/processed/test_hogged.txt'


In [3]:
train_df = pd.read_csv(train_csv_path, delimiter = ',', header='infer')
print('train_df listo')
test_df = pd.read_csv(test_csv_path, delimiter = ',', header='infer')
print('test_df listo')

train_feats_df = pd.read_csv(train_hoged_imgs_csv_path, delimiter = ',', header='infer')
print('train_feats_df listo')
test_feats_df = pd.read_csv(test_hoged_imgs_csv_path, delimiter = ',', header='infer')
print('test_feats_df listo')

train_df listo
test_df listo
train_feats_df listo
test_feats_df listo


In [22]:
train_df['features'] = train_feats_df['features']
test_df['features'] = test_feats_df['features']

In [25]:
numpy.array(train_df['features'][89].replace('[', '').replace(']', '').split(', ')).astype(float)

array([0.57426389, 0.00162015, 0.00409868, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
train_df['features'] = train_df['features'].apply(lambda x: numpy.array(x.replace('[', '').replace(']', '').split(', ')).astype(float) if isinstance(x, str) else x )
train_df.info(verbose=True)
train_df.head(10)

In [27]:
test_df['features'] = test_df['features'].apply(lambda x: numpy.array(x.replace('[', '').replace(']', '').split(', ')).astype(float))
test_df.info(verbose=True)
test_df.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1531 entries, 0 to 1530
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1531 non-null   int64 
 1   patient_id  1531 non-null   object
 2   image_id    1531 non-null   object
 3   disease     1531 non-null   object
 4   method      1531 non-null   object
 5   ancho       1531 non-null   int64 
 6   alto        1531 non-null   int64 
 7   features    1531 non-null   object
dtypes: int64(3), object(5)
memory usage: 95.8+ KB


,Unnamed: 0,patient_id,image_id,disease,method,ancho,alto,features
0,0,8,SARS-10.1148rg.242035193-g04mr34g05x-Fig5-day9...,pneumonia,cohen,1800,1593,"[0.008726461910522227, 0.0, 0.0, 0.07817275923..."
1,1,31,streptococcus-pneumoniae-pneumonia-temporal-ev...,pneumonia,cohen,1534,1517,"[0.3270144848975927, 0.0, 0.018498732468792765..."
2,2,31,streptococcus-pneumoniae-pneumonia-temporal-ev...,pneumonia,cohen,1534,1517,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,31,streptococcus-pneumoniae-pneumonia-temporal-ev...,pneumonia,cohen,1534,1517,"[0.2886060421081393, 0.3232476152184495, 0.326..."
4,4,31,streptococcus-pneumoniae-pneumonia-temporal-ev...,pneumonia,cohen,1534,1517,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,5,19,1-s2.0-S0929664620300449-gr2_lrg-a.jpg,COVID-19,cohen,1165,1163,"[0.014900795920243714, 0.0, 0.0, 0.0, 0.0, 0.0..."
6,6,19,1-s2.0-S0929664620300449-gr2_lrg-b.jpg,COVID-19,cohen,1170,1161,"[0.3953821145770147, 0.0010726572028767506, 0...."
7,7,19,1-s2.0-S0929664620300449-gr2_lrg-c.jpg,COVID-19,cohen,1165,1165,"[0.0020656651995510597, 0.0, 0.001460645845265..."
8,8,19,1-s2.0-S0929664620300449-gr2_lrg-d.jpg,COVID-19,cohen,1166,1161,"[0.7068614702148582, 0.01351142425733007, 0.0,..."
9,9,20,covid-19-pneumonia-15-PA.jpg,COVID-19,cohen,2800,3368,"[0.1303204097606551, 0.004437758813351989, 0.0..."


In [ ]:
print(len(train_df['features'][22]))
print(len(train_df['features'][55]))
print(len(train_df['features'][677]))
print(len(train_df['features'][9000]))
print(len(train_df['features'][10002]))
print(len(train_df['disease']))
print(len(train_df['features']))

In [ ]:
start = time.time()

clf = svm.SVC()

clf.fit(train_df['features'], train_df['disease'])

y_pred = clf.predict(test_df['features'])

end = time.time()
print(round(end - start, 2), "segundos")
print("Accuracy: "+str(accuracy_score(test_df['disease'], y_pred)))
print('\n')
print(classification_report(test_df['disease'], y_pred))